In [ ]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load the preprocessed data
wine_df = pd.read_csv() #'preprocessed_wine_data.csv')

external_stylesheets = [
    {
        "href": "assets/style.css",        
        "rel": "stylesheet",
    },
]
